In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
##############################################################################################################

def get_arrays(limits, image_path, pixel_scale, center):
    picture = cv2.imread(image_path)
    picture = cv2.cvtColor(picture, cv2.COLOR_BGR2GRAY)
    picture = picture[limits[0]:limits[1],limits[2]:limits[3]]
    #limits = [min_y, max_y, min_x, max_x]
    #picture[max_y:min_y, max_x:min_x]
    picture = picture.sum(axis=0)/len(picture[:,0])

    axis = np.arange(len(picture))
    axis = (axis - (center-limits[2]))/pixel_scale
    return axis, picture

def sinc_fit(x, A, B):
    return A*np.square(np.sinc(B*x))

def add_graphic(index, distance, intensity, fig, axes, entre_puntos, cotas, factor_A):

    fit_A =max( intensity )

   # maxima, properties_M = find_peaks(-1.0*intensity, distance=entre_puntos, height=(-1.0*cotas[0],0.0) )
    maxima, properties_M = find_peaks(-intensity, distance=entre_puntos)

    intensity_max = intensity[maxima]
    distance_max = distance[maxima]
#cotas cota_arriba, cota_derecha, cota_izquierda
    intensity_max = intensity_max[(distance_max < cotas[1]) & (distance_max > cotas[2]) ]
    distance_max =  distance_max[(distance_max < cotas[1]) & (distance_max > cotas[2]) ]
    distance_max =  distance_max[intensity_max < cotas[0] ]
    intensity_max =  intensity_max[intensity_max < cotas[0] ]

    distance_max = np.append(distance_max, 0.0)
    intensity_max = np.append(intensity_max, fit_A)

    distance_max_sort = np.sort(distance_max)

    space_betwen = distance_max_sort[1:] - distance_max_sort[:-1]
    spacing = np.mean(space_betwen)

    axes[index].plot(distance, fit_A*factor_A*np.square(np.sinc(distance/spacing)), marker = '.', markersize = '2', color='green', label = 'Ajuste')
    axes[index].plot(distance_max, intensity_max, 'ro')
    axes[index].plot(distance, intensity, '--', color='black', label='Medidas')
    print(f"Los parametros del ajuste son: B = {spacing}")
    axes[index].set_title("Intensidad a lo largo de una región horizontal")
    axes[index].set_xlabel("Posición [cm]")
    axes[index].set_ylim([0, min(  max(intensity), max(fit_A*factor_A*np.square(np.sinc(np.pi*distance/spacing))))])
    axes[index].set_ylabel("Intensidad (0-255)")
    axes[index].legend()

##############################################################################################################
ubicacion_base = "Rendija_3/"
la_info = pd.read_csv( ubicacion_base + "data3.csv", header=None, names=['parametro','valor_0', 'valor_1', 'valor_2'])
print(la_info)

escala = la_info["valor_0"][0]/la_info["valor_0"][1] #pixels/cm

limits_0 = [int(la_info["valor_0"][4]), int(la_info["valor_0"][5])
          , int(la_info["valor_0"][2]), int(la_info["valor_0"][3])]
limits_1 = [int(la_info["valor_1"][4]), int(la_info["valor_1"][5])
          , int(la_info["valor_1"][2]), int(la_info["valor_1"][3])]
limits_2 = [int(la_info["valor_2"][4]), int(la_info["valor_2"][5])
          , int(la_info["valor_2"][2]), int(la_info["valor_2"][3])]
#limits = [min_y, max_y, min_x, max_x]


center = [int(la_info["valor_0"][6]), int(la_info["valor_1"][6]), int(la_info["valor_2"][6])]

entre_puntos_0 = int(la_info["valor_0"][8])
entre_puntos_1 = int(la_info["valor_1"][8])
entre_puntos_2 = int(la_info["valor_2"][8])

cotas_0 = [la_info["valor_0"][9], la_info["valor_0"][10], la_info["valor_0"][11] ]
cotas_1 = [la_info["valor_1"][9], la_info["valor_1"][10], la_info["valor_1"][11] ]
cotas_2 = [la_info["valor_2"][9], la_info["valor_2"][10], la_info["valor_2"][11] ]
#cotas cota_arriba, cota_derecha, cota_izquierda

factorA_0 = la_info["valor_0"][12]
factorA_1 = la_info["valor_1"][12]
factorA_2 = la_info["valor_2"][12]

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6,12))

la_ubicacion = ubicacion_base + 'pico_1.jpeg'
distance, intensity = get_arrays(limits_0, la_ubicacion, escala, center[0])
add_graphic(0, distance, intensity, fig, axes, entre_puntos_0, cotas_0, factorA_0)

la_ubicacion = ubicacion_base + 'pico_2.jpeg'
distance, intensity = get_arrays(limits_1, la_ubicacion, escala, center[1])
add_graphic(1, distance, intensity, fig, axes, entre_puntos_1, cotas_1, factorA_1)

la_ubicacion = ubicacion_base + 'pico_3.jpeg'
distance, intensity = get_arrays(limits_2, la_ubicacion, escala, center[2])
add_graphic(2, distance, intensity, fig, axes, entre_puntos_2, cotas_2, factorA_2)

plt.tight_layout()
plt.show()
